# Code Breaking Part : Hill Cipher

The Hill Cipher used matrix multiplication to encrypt messages

The key for a Hill cipher is an $n \times n$ matrix of integers
- Easy Hill Ciphers are $2\times2$
ex) $$\begin{bmatrix} k_1 & k_2\\
k_3 & k_4 \end{bmatrix} $$ 

To encipher a message we start by breaking it into chunks of $n$ letters. eg) chunks of 2 letters for a $2\times2$ Hill cipher



In [1]:
from Resources.Functions import *
import numpy.linalg as linalg
from sympy import matrices as M

## Step 1) Encrypt the message

To encipher a message we start by breaking it into chunks of $n$ 
letters and then translate each letter into a number according to its place in the alphabet
- Ex) chunks of 2 letters for a $2\times2$ Hill cipher
- Ex) a = 0

We then write these chunks as vectors and multiply them against the key (which is a matrix).  

This results in another vector which we then take mod 26 of


$$ K \begin{bmatrix} p_1 \\
p_2\end{bmatrix} =  \begin{bmatrix} c_1 \\
c_2\end{bmatrix} \mod(26) $$ 

$$\begin{bmatrix} k_1 & k_2\\
k_3 & k_4 \end{bmatrix} \begin{bmatrix} p_1 \\
p_2\end{bmatrix} =  \begin{bmatrix} c_1 \\
c_2\end{bmatrix} \mod(26) $$ 

$$ k_1 p_1 + k_2 p_2 = c_1 \mod(26) $$

$$ k_3 p_1 + k_4 p_2 = c_2 \mod(26) $$


However, not every possible key is actually a useable key. 
In order for the key  to be useable it needs to have a  modular inverse. 

> In order to be a usable key, the matrix must have a non-zero determinant which is coprime to the length of the alphabet   
(https://crypto.interactive-maths.com/hill-cipher.html)

Ie  $$ d = k_1 k_4 - k_2 k_3 \neq 0 $$

and d and len(alphabet) are coprime. Meaning the greatest common demoninator of $d$ and len(alphabet) is 1


In [2]:
def Incrypt_2x2Hill( plainText, key, plainAlphabet=englishAlphabet ):
    '''
    **Incrypt a 2x2 Hill Cipher**
    
    key is a list of key-matrix elements in the order:
    key = [k_1, k_2, k_3, k_4] = [ k_1  k_2 ]
                                 [ k_3  k_4 ]
        
    You can customize the encryption further by reordering the alphabet
        - ex) Reverse the alphabet: englishAlphabet[::-1]
    '''
    k1, k2, k3, k4 = np.array(key).flatten()
    
    if len(plainText)%2 != 0:
        print('Invalid plain text length.  The length of the plain text must be a multiple of 2.')
        return    
    if k1*k4 - k2*k3 == 0 or gcd(k1*k4 - k2*k3, len(plainAlphabet)) != 1:
        print('Invalid key.  The determinant of the key must be non-zero and coprime to the length of the alphabet.')
        return 

    
    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }
    
    cipherText = ''
    # Read the message in groups of vectors of length 2
    for i in range(0,len(plainText), 2):
        
        p1, p2 = [ letter2num[letter] for letter in list(plainText[i:i+2]) ]
        
        c1 = num2letter[(k1*p1 + k2*p2)%len(plainAlphabet)]
        c2 = num2letter[(k3*p1 + k4*p2)%len(plainAlphabet)]
        
        cipherText += c1 + c2
    
    return cipherText

In [3]:
def Incrypt_Hill( plainText, key, plainAlphabet=englishAlphabet ):
    '''
    **Incrypt a nxn Hill Cipher**
    
    key is a nd array of key-matrix elements in the order:
    key = [ [k_1, k_2], [k_3, k_4] ] = [ k_1  k_2 ]
                                       [ k_3  k_4 ]
        
    You can customize the encryption further by reordering the alphabet
        - ex) Reverse the alphabet: englishAlphabet[::-1]
    '''
    d = int(np.round(linalg.det(key)))
    
    if len(plainText)%len(key) != 0:
        print('Invalid plain text length.  The length of the plain text must be a multiple of {}.'.format(len(key)))
        return    
    if d == 0 or gcd(d, len(plainAlphabet)) != 1:
        print('Invalid key.  The determinant of the key must be non-zero and coprime to the length of the alphabet.')
        return 

    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }
    
    cipherText = ''
    # Read the message in groups of vectors of length n
    for i in range(0,len(plainText), len(key)): 
        p = [ letter2num[letter] for letter in list(plainText[i:i+len(key)]) ]
        c = np.matmul(key, p)    
        cipherText += ''.join([ num2letter[i%len(plainAlphabet)] for i in c  ])
    
    return cipherText

In [8]:
Incrypt_Hill( 'HELLOWORLD', [[1,7],[3,4]] )

'JLKZMADGGT'

In [9]:
Incrypt_2x2Hill( 'HELLOWORLD', [[1,7],[3,4]])

'JLKZMADGGT'

## Step 2) Decrypt the message

We want to solve the incryption equation for $\vec{c}$
$$ K \begin{bmatrix} p_1 \\
p_2\end{bmatrix} =  \begin{bmatrix} c_1 \\
c_2\end{bmatrix} \mod(26) $$ 

$$ K^{-1} K \begin{bmatrix} p_1 \\
p_2\end{bmatrix} =  K^{-1}\begin{bmatrix} c_1 \\
c_2\end{bmatrix} \mod(26) $$ 


$$ \begin{bmatrix} p_1 \\
p_2\end{bmatrix} =  K^{-1}\begin{bmatrix} c_1 \\
c_2\end{bmatrix} \mod(26) $$ 


... where 
$$  K^{-1} = d^{-1} \begin{bmatrix} k_4 & -k_2\\
-k_3 & k_1 \end{bmatrix} \mod(26) $$

...where $d$ is the determinant of K in  mod 26
$$ d = k_1 k_4 - k_2 k_3 \mod(26) $$
and  
$$ d^{-1}d = 1 \mod(26) $$


therefore:



In [4]:
def Decrypt_2x2Hill( cipherText, key, plainAlphabet=englishAlphabet):
    '''
    **Decrypt a 2x2 Hill Cipher**
    
    key is a list of key-matrix elements in the order:
    key = [k_1, k_2, k_3, k_4] = [ k_1  k_2 ]
                                 [ k_3  k_4 ]
        
    You can customize the encryption further by reordering the alphabet
        - ex) Reverse the alphabet: englishAlphabet[::-1]
    ''' 
    k1, k2, k3, k4 = np.array(key).flatten()
    
    if len(cipherText)%2 != 0:
        print('Invalid cipher text length.  The length of the cipher text must be a multiple of 2.')
        return 
    if k1*k4 - k2*k3 == 0 or gcd(k1*k4 - k2*k3, len(plainAlphabet)) != 1:
        print('Invalid key.  The determinant of the key must be non-zero and coprime to the length of the alphabet.')
        return 
    
    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }

    dInvrt = pow(int((k1*k4 - k2*k3) % len(plainAlphabet)), -1,len(plainAlphabet))
    
    KInvrt_1 =  dInvrt*k4 % len(plainAlphabet)
    KInvrt_2 = -dInvrt*k2 % len(plainAlphabet)
    KInvrt_3 = -dInvrt*k3 % len(plainAlphabet)
    KInvrt_4 =  dInvrt*k1 % len(plainAlphabet)
    
#     print(dInvrt)
#     print(KInvrt_1,KInvrt_2,KInvrt_3,KInvrt_4)
    
    plainText = ''
    # Read the message in groups of vectors of length 2
    for i in range(0,len(cipherText), 2):
        c1, c2 = [ letter2num[letter] for letter in list(cipherText[i:i+2]) ]
        
        p1 = num2letter[(KInvrt_1*c1 + KInvrt_2*c2)%len(plainAlphabet)]
        p2 = num2letter[(KInvrt_3*c1 + KInvrt_4*c2)%len(plainAlphabet)]
    
        plainText += p1 + p2
    
    return plainText


In [5]:
def Decrypt_Hill( cipherText, key, plainAlphabet=englishAlphabet):
    '''
    **Incrypt a nxn Hill Cipher**
    
    key is a nd array of key-matrix elements in the order:
    key = [ [k_1, k_2], [k_3, k_4] ] = [ k_1  k_2 ]
                                       [ k_3  k_4 ]
        
    You can customize the encryption further by reordering the alphabet
        - ex) Reverse the alphabet: englishAlphabet[::-1]
    '''
    d = int(np.round(linalg.det(key)))
    
    if len(cipherText)%len(key) != 0:
        print('Invalid plain text length.  The length of the plain text must be a multiple of {}.'.format(len(key)))
        return 
    if d == 0 or gcd(d, len(plainAlphabet)) != 1:
        print('Invalid key.  The determinant of the key must be non-zero and coprime to the length of the alphabet.')
        return 
    
    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }

    dInvrt = pow( (d % len(plainAlphabet)), -1,len(plainAlphabet))
    
    keyInvrt = dInvrt* np.array(M.Matrix(key).adjugate()) % len(plainAlphabet)
    
    plainText = ''
    # Read the message in groups of vectors of length n
    for i in range(0,len(cipherText), len(key)):
        c = [ letter2num[letter] for letter in list(cipherText[i:i+len(key)]) ]
        p = np.matmul(keyInvrt, c)    
        plainText += ''.join([ num2letter[i%len(plainAlphabet)] for i in p  ])
    
    return plainText


## Step 3) Breaking the 2x2 Hill Cipher

The Hill Cipher is a polyalbahetic substitution cipher which may yeild some peaks on a periodic IOC calculation. The locations of the peaks may be somewhat indicative of the size of the key matrix, however, we know that the text length must be a multiple of the key size. 





In [6]:
def Break_2x2HillBruteForce( cipherText,plainAlphabet=englishAlphabet, knownLetterFrequencies=letterFrequencies_English ):
    '''
    **Break a 2x2 Hill Cipher with a Brute Force Attack **
    
    The 2x2 Hill cipher has few enough possible keys that a brute force attack 
    is possible (though not always fast).  Here every possible key is checked and 
    the resulting decryption is assessed using Chi-Squared Statistics.
    
    -Please remove all non-text characters before using
    '''
    
    if len(cipherText)%2 != 0:
        print('Invalid cipher text length.  The length of the cipher text must be a multiple of 2.')
        return 

    bestKey = []
    bestScore = 99e99
    
    possibleKeys = []
    for key in itertools.permutations(list(range(0,len(plainAlphabet))), 4):
        k1, k2, k3, k4 = np.array(key).flatten()
        if k1*k4 - k2*k3 != 0 and gcd(k1*k4 - k2*k3, len(plainAlphabet)) == 1:
            possibleKeys.append(key)
    
    for key in possibleKeys:
        decryptTest = Decrypt_2x2Hill( cipherText, key, plainAlphabet)
        score = ChiSquaredStatistic(decryptTest, knownLetterFrequencies)
        if score < bestScore:
            bestScore = score
            bestKey = key
    
    return bestKey, Decrypt_2x2Hill( cipherText, bestKey, plainAlphabet)

### Bigram Matching

The most common bigrams in English, in order, are: 
'TH', 'HE', 'IN', 'ER', 'AN', 'RE

- Let a bigram in *cipher text* be $b_1 = b_{1,1} b_{1,2}$
- Let a different bigram in *cipher text* be $b_2 = b_{2,1} b_{2,2}$


- Let a bigram in *plain text* be $m_1 = m_{1,1} m_{1,2}$ 
    - ex) $m_1$ = 'TH'
- Let a different bigram in *plain text* be $m_2 = m_{2,1} m_{2,2}$
    - ex) $m_2$ = 'HE'

Lets say that we think: $m_1 \to b_1$ and  $m_2 \to b_2$

Then, since:
$$ K P = C \mod (26) $$

$$ K = C P^{-1} \mod(26) $$

... where 
$$ C =  \begin{bmatrix} b_{1,1} & b_{2,1}\\
b_{1,2} & b_{2,2} \end{bmatrix} $$
and
$$ P =  \begin{bmatrix} m_{1,1} & m_{2,1}\\
m_{1,2} & m_{2,2} \end{bmatrix} $$

Recall 
$$ P^{-1} = d^{-1} \begin{bmatrix} m_{2,2} & -m_{2,1}\\
-m_{1,2} & m_{1,1} \end{bmatrix}   $$
and $$ d^{-1}d = 1 \mod(26) $$


$$ K = \begin{bmatrix} b_{1,1} & b_{2,1}\\
b_{1,2} & b_{2,2} \end{bmatrix}  \begin{bmatrix} d^{-1}m_{2,2} & -d^{-1}m_{2,1}\\
-d^{-1}m_{1,2} & d^{-1}m_{1,1} \end{bmatrix} \mod(26) $$

$$ k1 =  b_{1,1} d^{-1}m_{2,2} - b_{2,1} d^{-1}m_{1,2} \mod(26)$$
$$ k2 = -b_{1,1} d^{-1}m_{2,1} + b_{2,1} d^{-1}m_{1,1} \mod(26)$$
$$ k3 =  b_{1,2} d^{-1}m_{2,2} - b_{2,2} d^{-1}m_{1,2} \mod(26)$$
$$ k4 = -b_{1,2} d^{-1}m_{2,1} + b_{2,2} d^{-1}m_{1,1} \mod(26)$$

In [37]:
def Break_2x2HillBigramMatching( cipherText,b1,m1,b2,m2, plainAlphabet=englishAlphabet):
    '''
    **Break a 2x2 Hill Cipher with Bigram Matching**
    
    bigram1 = b1 = a bigram in the cipher text
    match1  = m1 = the likley plain text version of b1
        - If you choose b1 to be the most common bigram in the
        cipher text, then m1 = 'TH'  would be logical
    
    bigram2 = b2 = another bigram in the cipher text
    match2  =  m2 = the likley plain text version of b2
            - If you choose b2 to be the second most common bigram in the
        cipher text, then m2 = 'HE'  would be logical
    
    -Please remove all non-text characters before using
    '''
    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }
    
    m11, m12 = [ letter2num[c] for c in list(m1)]
    m21, m22 = [ letter2num[c] for c in list(m2)]
    
    b11, b12 = [ letter2num[c] for c in list(b1)]
    b21, b22 = [ letter2num[c] for c in list(b2)]
    
    if len(cipherText)%2 != 0:
        print('Invalid cipher text length.  The length of the cipher text must be a multiple of 2.')
        return 
    
#     if m11*m22 - m21*m12 == 0 or gcd(m11*m22 - m21*m12, len(plainAlphabet)) != 1:
# #         print('Invalid key.  The determinant of the key must be non-zero and coprime to the length of the alphabet.')
#         return 
    
    
    dInvert = pow(int((m11*m22 - m21*m12) % len(plainAlphabet)), -1,len(plainAlphabet))  
    print(dInvert)
    
    print('P = ')
    print(m11, m21)
    print(m12, m22)
    
    print('\n')
    
    print('C = ')
    print(b11, b21)
    print(b12, b22)
       
    print('\n')
    
    print('P^-1 = ')
    print(dInvert*m22 % len(plainAlphabet), -dInvert*m21 % len(plainAlphabet))
    print(-dInvert*m12 % len(plainAlphabet), dInvert*m11 % len(plainAlphabet))
    
    k1 = dInvert*( b11*m22 - b21*m12 ) % len(plainAlphabet) 
    k2 = dInvert*(-b11*m21 + b21*m11 ) % len(plainAlphabet)
    k3 = dInvert*( b12*m22 - b22*m12 ) % len(plainAlphabet)
    k4 = dInvert*(-b12*m21 + b22*m11 ) % len(plainAlphabet)
    
    key = [k1,k2,k3,k4]

    if k1*k4 - k2*k3 != 0 and gcd(k1*k4 - k2*k3, len(plainAlphabet)) == 1:
        return key, Decrypt_2x2Hill( cipherText, key, plainAlphabet)
    else:
        return None

In [38]:
def Break_2x2HillCrib( cipherText,crib, plainAlphabet=englishAlphabet, knownLetterFrequencies=letterFrequencies_English):
    '''
    ** Break a 2x2 Hill Cipher with a Crib **
    
    The crib message is "dragged" across the cipher text
        - We place the crib at different positions along the cipher text
        and extract potential bigram matches 
    
        - The crib must be 5 letters long
        
        - Try the most common 5grams: 'OFTHE' 'INTHE' 'ATION' 'THERE' 'TOTHE' 'ANDTH'
    '''
    if len(cipherText)%2 != 0:
        print('Invalid cipher text length.  The length of the cipher text must be a multiple of 2.')
        return 
    
    bestScore = 99e99
    bestKey = [ ]
    
    for i in range(0, len(cipherText)-len(crib)):
        if i % 2 == 0:
            m1 = crib[:2]
            b1 = cipherText[i:i+2]
            m2 = crib[2:4]
            b2 = cipherText[i+2:i+4]
        else:
            m1 = crib[1:3]
            b1 = cipherText[i+1:i+3]
            m2 = crib[3:5]
            b2 = cipherText[i+3:i+5]
            
        BreakTest = Break_2x2HillBigramMatching( cipherText,b1,m1,b2,m2, plainAlphabet)
        if BreakTest != None:
            print('me')
            key, decryptTest  = BreakTest
            score = ChiSquaredStatistic(decryptTest)
            if score < bestScore:
                print('me')
                bestScore = score
                bestKey = key
    
#     return bestKey, Decrypt_2x2Hill( cipherText, bestKey, plainAlphabet)

In [10]:
def Break_HillngramMatching(cipherText,b,m, plainAlphabet=englishAlphabet):
    '''
    **Break a nxn Hill Cipher with ngram Matching**
    
    bigram = b = a list of ngrams in the cipher text
    match  = m = a list of the likley plain text versions of b

    -Please remove all non-text characters before using
    '''
    
    
    letter2num = { c.upper():i for i , c in enumerate(plainAlphabet) }
    num2letter = { i:c.upper() for i , c in enumerate(plainAlphabet) }
    
    P = np.array([ [ letter2num[c] for c in list(ngram) ] for ngram in m]).T
    C = np.array([ [ letter2num[c] for c in list(ngram) ] for ngram in b]).T
    print('C: {}'.format(C))
    print('P: {}'.format(P))
    
   
    d = int(np.round(linalg.det(P)))
    dInvrt = pow( (d % len(plainAlphabet)), -1,len(plainAlphabet))
    PInvrt = dInvrt* np.array(M.Matrix(P).adjugate()) % len(plainAlphabet)
    print(PInvrt)
    
    key = np.array(np.matmul(C, PInvrt) % len(plainAlphabet), dtype=int)


    if d != 0 and gcd(d, len(plainAlphabet)) == 1:
        return key , Decrypt_Hill( cipherText, key, plainAlphabet)
    else:
        return None

In [11]:
def Break_HillCrib( cipherText,crib,n, plainAlphabet=englishAlphabet, knownLetterFrequencies=letterFrequencies_English):
    '''
    ** Break a nxn Hill Cipher with a Crib **
    
    The crib message is "dragged" across the cipher text
        - We place the crib at different positions along the cipher text
        and extract potential bigram matches 
    '''
    
    bestScore = 99e99
    bestKey = [ ]
    
    for i in range(0, len(cipherText)-len(crib)):

        offset = i % n
        print('i: {}, offset: {}'.format(i, offset))
        
        if offset == 0:
            k = 0
        else:
            k = n - offset
        
        b = [ cipherText[i + k + j*n: i + k + (j+1)*n] for j in range(n)]

        m = [ crib[k + j*n: k + (j+1)*n] for j in range(n)]

        BreakTest = Break_HillngramMatching( cipherText,b,m, plainAlphabet)
        
        if BreakTest != None:
            key, decryptTest  = BreakTest
            score = ChiSquaredStatistic(decryptTest)
            
            if score < bestScore:
                bestScore = score
                bestKey = key
                
    return bestKey, Decrypt_Hill( cipherText, bestKey, plainAlphabet)

In [16]:
plainText = 'THISMESSAGEISATEST'
key = [[1, 13], [4, 23]]

cipherText = 'GDIEMKSSAIESSUTMFP'
print(Incrypt_Hill(plainText, key  ))
print(Decrypt_Hill(cipherText, key))

print(Incrypt_2x2Hill(plainText, key))
print(Decrypt_2x2Hill(cipherText, key))

GDIEMKSSAIESSUTMFP
THISMESSAGEISATEST
GDIEMKSSAIESSUTMFP
THISMESSAGEISATEST


In [39]:
Break_2x2HillCrib(cipherText,'THISM')


# Break_2x2HillBigramMatching(cipherText,'GD','TH','DI','HI')

ValueError: base is not invertible for the given modulus